In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# 시간
import arrow
import os
# 3.11까지 호환 개발됨
from webdriver_manager.chrome import ChromeDriverManager
import requests

In [2]:
import json
import re
import urllib.parse

In [3]:
# create driver
chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument("window-size=1920,1024")
chrome_prefs = {
    "credentials_enable_service": False,
    "profile.password_manager_enabled": False,
    "profile.password_manager_leak_detection": False,  # <== 중요!
}
chrome_options.add_experimental_option("prefs", chrome_prefs)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

In [4]:
login_url = "https://www.incheoncc.com:1436/login/login.asp?returnurl=/pagesite/reservation/live.asp?"
login_id = os.environ.get("LOGIN_ID")
login_pw = os.environ.get("LOGIN_PW")
driver.get(login_url)
id_element = driver.find_element(value="login_id")
pw_element = driver.find_element(value="login_pw")
id_element.send_keys(login_id)
pw_element.send_keys(login_pw)
driver.find_element(value="bt_login",by=By.CLASS_NAME).click()
# alert 뜸 -> # UnexpectedAlertPresentException
# 엘리먼트가 없다면 NoSuchElementException

In [6]:
# Selenium에서 세션 쿠키 가져오기
selenium_cookies = driver.get_cookies()

# requests 세션 초기화
session = requests.Session()

# Selenium에서 가져온 세션 쿠키를 requests 세션에 추가
for cookie in selenium_cookies:
    session.cookies.set(cookie['name'], cookie['value'])


In [7]:
# range time
from dataclasses import dataclass

@dataclass
class TimePoint:
    hour: int
    minute: int

    def strf_hhmm(self):
        return f"{self.hour:02d}{self.minute:02d}"
        

@dataclass
class TimeRange: 
    start: TimePoint
    end: TimePoint
    priority: TimePoint
    

In [8]:
from typing import List

def get_all_timepoints_in_range(time_range: TimeRange) -> List[TimePoint]:
    result = []
    start_minutes = time_range.start.hour * 60 + time_range.start.minute
    end_minutes = time_range.end.hour * 60 + time_range.end.minute
    for m in range(start_minutes, end_minutes + 1):
        hour = m // 60
        minute = m % 60
        result.append(TimePoint(hour, minute))
    return result

In [9]:
def sort_by_priority(timepoints: List[TimePoint], priority: TimePoint) -> List[TimePoint]:
    def diff(tp: TimePoint):
        return abs((tp.hour * 60 + tp.minute) - (priority.hour * 60 + priority.minute))
    return sorted(timepoints, key=diff)

In [10]:
# ... 위의 TimePoint, TimeRange, 함수 정의 ...

# 예시
tr = TimeRange(
    start=TimePoint(5, 0),
    end=TimePoint(6, 0),
    priority=TimePoint(5, 30)
)

all_times = get_all_timepoints_in_range(tr)
sorted_times = sort_by_priority(all_times, tr.priority)

for tp in sorted_times:
    print(tp.strf_hhmm())

0530
0529
0531
0528
0532
0527
0533
0526
0534
0525
0535
0524
0536
0523
0537
0522
0538
0521
0539
0520
0540
0519
0541
0518
0542
0517
0543
0516
0544
0515
0545
0514
0546
0513
0547
0512
0548
0511
0549
0510
0550
0509
0551
0508
0552
0507
0553
0506
0554
0505
0555
0504
0556
0503
0557
0502
0558
0501
0559
0500
0600


In [20]:
point_date="20250613"
point_id_out_in = "2" # # OUT(1),IN(2)
point_time = "0530"
# environ
hand_tel1=os.environ.get("HAND_TEL1")
hand_tel2=os.environ.get("HAND_TEL2")
hand_tel3=os.environ.get("HAND_TEL3")


reserve_ok_url = "https://www.incheoncc.com:1436/GolfRes/onepage/real_resok.asp"

# POST 요청에 사용할 데이터
form_data ={
    "cmd" : "ins",
    "cmval" : "0",
    "cmrtype" : "N",
    "calltype": "AJAX",
    "gonexturl" : "/GolfRes/onepage/my_golfreslist.asp",
    "pointdate" : point_date,
    "openyn" : "1",
    "dategbn" : "6",
    "pointid" : point_id_out_in,
    "pointtime" : point_time,
    "flagtype" : "I",
    "punish_cd" : "UNABLE",
    "self_r_yn" : "N",
    "res_gubun" : "N",
    "usrmemcd" : "12",
    "memberno" : "12061000",
    "hand_tel1" : hand_tel1,
    "hand_tel2" : hand_tel2,
    "hand_tel3" : hand_tel3,   
}

In [21]:
form_data

{'cmd': 'ins',
 'cmval': '0',
 'cmrtype': 'N',
 'calltype': 'AJAX',
 'gonexturl': '/GolfRes/onepage/my_golfreslist.asp',
 'pointdate': '20250613',
 'openyn': '1',
 'dategbn': '6',
 'pointid': '2',
 'pointtime': '0530',
 'flagtype': 'I',
 'punish_cd': 'UNABLE',
 'self_r_yn': 'N',
 'res_gubun': 'N',
 'usrmemcd': '12',
 'memberno': '12061000',
 'hand_tel1': '010',
 'hand_tel2': '7430',
 'hand_tel3': '0955'}

In [135]:
# direct
# 참고 사이트: https://13months.tistory.com/484
# 요청을 보낼 URL

for idx, point_time in enumerate(sorted_times,start=1):
    form_data["pointtime"] = point_time.strf_hhmm()
    response = session.post(reserve_ok_url, data=form_data)
    if "OK" in response.text:
        print(f"{idx}번째 시도, {form_data["pointtime"]} 예약 성공")
        break
    elif "오류" in response.text:
        print(f"{idx}번째 시도, {form_data["pointtime"]} 예약 실패, 없는 시간")
    elif "동시예약" in response.text:
        print(f"{idx}번째 시도, {form_data["pointtime"]} 예약 실패, 동시예약으로 인한 실패")
    elif "다른 곳에서 회원님의 아이디로 로그인 되었습니다." in response.text:
        print(f"{idx}번째 시도, {form_data["pointtime"]} 예약 실패, 다른 곳에서 로그인 함(세션 불일치)")

1번째 시도, 0530 예약 성공


In [22]:
response.text

NameError: name 'response' is not defined

In [23]:
# 1. \r, \n, \t 등 제거
cleaned = response.text.strip()

# 2. JSON 파싱
data = json.loads(cleaned)

# 3. %uXXXX → 한글로 변환
def decode_unicode_escapes(s):
    # %uXXXX → \uXXXX로 변환
    s = re.sub(r'%u([0-9A-Fa-f]{4})', r'\\u\1', s)
    # %XX → 문자로 변환
    s = urllib.parse.unquote(s)
    # \uXXXX → 실제 문자로 변환
    return s.encode('utf-8').decode('unicode_escape')

data['gomsg'] = decode_unicode_escapes(data['gomsg'])
print(data)

NameError: name 'response' is not defined